# GameBus Health Behavior Mining - Data Extraction

This notebook demonstrates how to extract data from the GameBus platform using the framework.

## Setup

First, let's set up our environment and import the necessary modules.

In [1]:
import sys
import os
import pandas as pd
import json
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Add the project root directory to the Python path
sys.path.append('..')

# Import project modules
from config.credentials import AUTHCODE
from config.paths import USERS_FILE_PATH, RAW_DATA_DIR
from src.extraction.gamebus_client import GameBusClient
from src.extraction.data_collectors import (
    LocationDataCollector, 
    MoodDataCollector,
    ActivityTypeDataCollector,
    HeartRateDataCollector,
    AccelerometerDataCollector,
    NotificationDataCollector
)
from src.utils.logging import setup_logging
from src.utils.file_handlers import load_json, save_json, load_csv, save_csv

## Initialize Logging

Set up logging for the notebook.

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG)
logger = setup_logging(log_level="INFO")
logger.info("Notebook initialized")

2025-05-15 11:22:38,810 - gamebus_health_mining - INFO - Notebook initialized


INFO:gamebus_health_mining:Notebook initialized


## Load GameBus Users

Load the GameBus users from the CSV file. If the file doesn't exist yet, we'll create it from the example file.

In [3]:
# Load the users
try:
    users_df = pd.read_csv(USERS_FILE_PATH, delimiter=';')
    display(users_df.head())
    print(f"Loaded {len(users_df)} users")
except Exception as e:
    # If users file doesn't exist yet, we need to create it
    print(f"Failed to load users file: {e}")
    print("Loading example users from the examples directory...")
    example_users_path = os.path.join('..', 'examples', 'GB-users.csv')
    users_df = pd.read_csv(example_users_path, delimiter=';')
    # Save to configured path
    os.makedirs(os.path.dirname(USERS_FILE_PATH), exist_ok=True)
    users_df.to_csv(USERS_FILE_PATH, sep=';', index=False)
    display(users_df.head())
    print(f"Loaded {len(users_df)} example users and saved to {USERS_FILE_PATH}")

,Username,Password
0,info+bm@gamebus.nl,password1234
1,m.i.da.graca.jorge.da.silva.ribeiro+test@tue.nl,password1234


Loaded 2 users


## Initialize GameBus Client

Create a GameBus client for API interactions.

In [4]:
# Initialize the GameBus client
client = GameBusClient(AUTHCODE)
print("GameBus client initialized")

GameBus client initialized


## Define Date Range (Optional)

Optionally define a date range for data extraction. If no dates are specified, all available data will be extracted.

In [5]:
# Set to True to enable date filtering
use_date_filter = True

if use_date_filter:
    # Define start date (inclusive)
    start_date = datetime.datetime(2025, 5, 9)  # Format: YYYY, MM, DD
    
    # Define end date (inclusive)
    end_date = datetime.datetime(2025, 7, 1)    # Format: YYYY, MM, DD
    
    print(f"Date range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
else:
    start_date = None
    end_date = None
    print("No date filtering applied - extracting all available data")

Date range: 2025-05-09 to 2025-07-01


## Select User

Select a user from the list to extract data for.

In [6]:
# Select a user
user_index = 0  # Change this to select a different user
selected_user = users_df.iloc[user_index]
username = selected_user['Username']
password = selected_user['Password']

print(f"Selected user: {username}")

Selected user: info+bm@gamebus.nl


## Authenticate User

Authenticate with the GameBus API and get the player token and ID.

In [7]:
# Get player token and ID
token = client.get_player_token(username, password)
player_id = client.get_player_id(token)

print(f"Player token: {token[:10]}..." if token else "Failed to get token")
print(f"Player ID: {player_id}" if player_id else "Failed to get player ID")

DEBUG:src.extraction.gamebus_client:Making POST request to: https://api-new.gamebus.eu/v2/oauth/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api-new.gamebus.eu:443
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "POST /v2/oauth/token HTTP/11" 200 None
INFO:src.extraction.gamebus_client:Token fetched successfully
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/users/current
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/users/current HTTP/11" 200 None
INFO:src.extraction.gamebus_client:Player ID fetched successfully


Player token: eyJhbGciOi...
Player ID: 107631


## Extract GPS Location Data

Extract GPS location data from GameBus. This includes latitude, longitude, altitude, speed, error, timestamp, and arm.

In [8]:
# Create a location data collector
location_collector = LocationDataCollector(client, token, player_id)

# Collect location data
location_data, location_file = location_collector.collect(start_date=start_date, end_date=end_date)

print(f"Collected {len(location_data)} location data points")
print(f"Data saved to {location_file}")

# Display a sample of the data
location_df = pd.DataFrame(location_data)
if len(location_df) > 0:
    display(location_df.head())

INFO:src.extraction.gamebus_client:Filtering data from: 2025-05-09 00:00:00 (timestamp: 1746741600000)
INFO:src.extraction.gamebus_client:Filtering data to: 2025-07-01 00:00:00 (timestamp: 1751320800000)
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=GEOFENCE&from=1746741600000&to=1751320800000&page=0
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=GEOFENCE&from=1746741600000&to=1751320800000&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-15 11:19:09
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 500 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=GEOFENCE&from=1746741600000&to=1751320800000&page=1
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /

Collected 15130 location data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_location.json


,LATITUDE,LONGITUDE,ALTIDUDE,SPEED,ERROR,TIMESTAMP,ARM,activity_id,date,gameDescriptor
0,200,200,0,0,0,1747300689,Arm 2,3358902,1747300749000,GEOFENCE
1,200,200,0,0,0,1747300628,Arm 2,3358901,1747300689000,GEOFENCE
2,200,200,0,0,0,1747300568,Arm 2,3358900,1747300629000,GEOFENCE
3,200,200,0,0,0,1747300508,Arm 2,3358897,1747300568000,GEOFENCE
4,200,200,0,0,0,1747300448,Arm 2,3358896,1747300508000,GEOFENCE


## Extract Mood Data

Extract mood logging data from GameBus. This includes valence_state_value, arousal_state_value, stress_state_value, and event_timestamp.

In [9]:
# Create a mood data collector
mood_collector = MoodDataCollector(client, token, player_id)

# Collect mood data
mood_data, mood_file = mood_collector.collect()

print(f"Collected {len(mood_data)} mood data points")
print(f"Data saved to {mood_file}")

# Display a sample of the data
mood_df = pd.DataFrame(mood_data)
if len(mood_df) > 0:
    display(mood_df.head())

DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=LOG_MOOD&page=0
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=LOG_MOOD&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-13 18:01:59
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 48 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=LOG_MOOD&page=1
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=LOG_MOOD&page=1 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:Total data points fetched: 48
INFO:src.extraction.data_collectors:Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_m

Collected 48 mood data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_mood.json


,VALENCE_STATE_VALUE,AROUSAL_STATE_VALUE,STRESS_STATE_VALUE,EVENT_TIMESTAMP,activity_id,date,gameDescriptor
0,6,6,4,1747151791916,3339567,1747152119000,LOG_MOOD
1,6,6,4,1747129592896,3338925,1747129729000,LOG_MOOD
2,6,4,7,1747118721807,3338630,1747119830000,LOG_MOOD
3,4,6,6,1747077705478,3338411,1747080469000,LOG_MOOD
4,2,4,6,1747054671443,3337821,1747064157000,LOG_MOOD


## Extract Activity Type Data

Extract activity type data from GameBus. This includes src, ts (timestamp), type, speed, steps, walks, runs, freq, distance, and cals.

In [10]:
# Create an activity type data collector
activity_collector = ActivityTypeDataCollector(client, token, player_id)

# Collect activity type data
activity_data, activity_file = activity_collector.collect()

print(f"Collected {len(activity_data)} activity data points")
print(f"Data saved to {activity_file}")

# Display a sample of the data
activity_df = pd.DataFrame(activity_data)
if len(activity_df) > 0:
    display(activity_df.head())

DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=0
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-13 18:35:53
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 500 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=1
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=1 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 1: 2025-05-12 14:33:37
INFO:src.extraction.gamebus_client:Successfully retrieved page 1 with 500 items
DEBUG:src.extraction.gamebus

Collected 2021 activity data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_activity_type.json


,src,ts,type,speed,steps,walks,runs,freq,distance,cals,activity_id,activity_date
0,p,1747153174727,NOT_MOVING,0.0,1410.0,1410.0,0.0,0.0,1037.589966,51.18,3339578,1747153175000
1,p,1747151764994,NOT_MOVING,0.0,1410.0,1410.0,0.0,0.0,1037.589966,51.18,3339564,1747151765000
2,p,1747151687987,NOT_MOVING,0.0,1410.0,1410.0,0.0,0.0,1037.589966,51.18,3339561,1747151688000
3,p,1747151285990,NOT_MOVING,0.0,1410.0,1410.0,0.0,0.0,1037.589966,51.18,3339552,1747151286000
4,p,1747150881534,NOT_MOVING,0.0,1410.0,1410.0,0.0,0.0,1037.589966,51.18,3339544,1747150882000


## Extract Heart Rate Data

Extract heart rate monitoring data from GameBus. This includes ts (timestamp), hr (heartrate), and pp.

In [11]:
# Create a heart rate data collector
heartrate_collector = HeartRateDataCollector(client, token, player_id)

# Collect heart rate data
heartrate_data, heartrate_file = heartrate_collector.collect()

print(f"Collected {len(heartrate_data)} heart rate data points")
print(f"Data saved to {heartrate_file}")

# Display a sample of the data
heartrate_df = pd.DataFrame(heartrate_data)
if len(heartrate_df) > 0:
    display(heartrate_df.head())

DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=0
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-13 18:35:53
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 500 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=1
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=1 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 1: 2025-05-12 14:33:37
INFO:src.extraction.gamebus_client:Successfully retrieved page 1 with 500 items
DEBUG:src.extraction.gamebus

Collected 906173 heart rate data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_heartrate.json


,ts,hr,pp,activity_id,activity_date
0,1747154093767,-3,0,3339629,1747154153000
1,1747154093829,-3,0,3339629,1747154153000
2,1747154093909,-3,0,3339629,1747154153000
3,1747154093989,-3,0,3339629,1747154153000
4,1747154094070,-3,0,3339629,1747154153000


## Extract Accelerometer Data

Extract accelerometer data from GameBus. This includes ts (timestamp), x axis, y axis, and z axis.

In [12]:
# Create an accelerometer data collector
accelerometer_collector = AccelerometerDataCollector(client, token, player_id)

# Collect accelerometer data
accelerometer_data, accelerometer_file = accelerometer_collector.collect()

print(f"Collected {len(accelerometer_data)} accelerometer data points")
print(f"Data saved to {accelerometer_file}")

# Display a sample of the data
accelerometer_df = pd.DataFrame(accelerometer_data)
if len(accelerometer_df) > 0:
    display(accelerometer_df.head())

DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=0
DEBUG:urllib3.connectionpool:Resetting dropped connection: api-new.gamebus.eu
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-13 18:35:53
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 500 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=1
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=TIZEN%28DETAIL%29&page=1 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 1: 2025-05-12 14:33:37
INFO:src.extraction.gamebus_cl

Collected 3489509 accelerometer data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_accelerometer.json


,ts,x,y,z,activity_id,activity_date
0,1747154093260,0.081356,-0.351745,9.698111,3339632,1747154153000
1,1747154093265,0.055035,-0.382852,9.686146,3339632,1747154153000
2,1747154093268,0.059821,-0.437887,9.655040,3339632,1747154153000
3,1747154093268,0.081356,-0.387637,9.698111,3339632,1747154153000
4,1747154093269,0.129212,-0.390030,9.674182,3339632,1747154153000


## Extract Notification Data

Extract notification data from GameBus. This includes action (e.g., received, read) and event_timestamp.

In [13]:
# Create a notification data collector
notification_collector = NotificationDataCollector(client, token, player_id)

# Collect notification data
notification_data, notification_file = notification_collector.collect()

print(f"Collected {len(notification_data)} notification data points")
print(f"Data saved to {notification_file}")

# Display a sample of the data
notification_df = pd.DataFrame(notification_data)
if len(notification_df) > 0:
    display(notification_df.head())

DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=NOTIFICATION%28DETAIL%29&page=0
DEBUG:urllib3.connectionpool:Resetting dropped connection: api-new.gamebus.eu
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=NOTIFICATION%28DETAIL%29&page=0 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:First item date on page 0: 2025-05-13 17:56:25
INFO:src.extraction.gamebus_client:Successfully retrieved page 0 with 121 items
DEBUG:src.extraction.gamebus_client:Making GET request to: https://api-new.gamebus.eu/v2/players/107631/activities?sort=-date&size=50&gds=NOTIFICATION%28DETAIL%29&page=1
DEBUG:urllib3.connectionpool:https://api-new.gamebus.eu:443 "GET /v2/players/107631/activities?sort=-date&size=50&gds=NOTIFICATION%28DETAIL%29&page=1 HTTP/11" 200 None
INFO:src.extraction.gamebus_client:Total data points fetched: 121
INFO:src.extractio

Collected 121 notification data points
Data saved to c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\data\raw\player_107631_notifications.json


,ACTION,EVENT_TIMESTAMP,activity_id,date,gameDescriptor
0,READ,1747151785023,3339566,1747151785000,NOTIFICATION(DETAIL)
1,RECEIVED,1747151766482,3339565,1747151766000,NOTIFICATION(DETAIL)
2,RECEIVED,1747140900599,3339260,1747140901000,NOTIFICATION(DETAIL)
3,READ,1747129585748,3338924,1747129586000,NOTIFICATION(DETAIL)
4,RECEIVED,1747129574715,3338923,1747129575000,NOTIFICATION(DETAIL)


## Summary

Let's summarize all the data we've collected.

In [14]:
# Create a summary of collected data
summary = {
    "Location": len(location_data),
    "Mood": len(mood_data),
    "Activity": len(activity_data),
    "Heart Rate": len(heartrate_data),
    "Accelerometer": len(accelerometer_data),
    "Notification": len(notification_data)
}

summary_df = pd.DataFrame({
    "Data Type": list(summary.keys()),
    "Count": list(summary.values())
})

# Display summary
display(summary_df)

print(f"\nTotal data points collected: {sum(summary.values())}")

,Data Type,Count
0,Location,13408
1,Mood,48
2,Activity,2021
3,Heart Rate,906173
4,Accelerometer,3489509
5,Notification,121



Total data points collected: 4411280


## Next Steps

Now that we have extracted the raw data from GameBus, the next steps would be:

1. **Preprocessing**: Clean and normalize the data, handle missing values, synchronize timestamps, etc.
2. **Activity Recognition**: Recognize human activities from the sensor data using machine learning techniques.
3. **OCEL Generation**: Transform the preprocessed data and recognized activities into the Object-Centric Event Log (OCEL) format for process mining.

These steps will be covered in the subsequent notebooks.